This notebook is only for testing. Maybe we can build a working model from this, and then we can start putting code into the "model" python module.

In [ ]:
import tensorflow as tf
import numpy as np
import time
import importlib
import os
import re

In [ ]:
import sys
sys.path.append('../py')
import cavitylearn.data
import cavitylearn.catalonet0

## Loading Data

In [ ]:
dataset_basedir = '/var/cavitylearn/dataset-test/'

In [ ]:
dataconfig = cavitylearn.data.DataConfig(classes=["SO4","GOL","EDO","HEM","NAG","PO4","ACT"], num_props=8, boxshape=[30, 30, 30])
with open(os.path.join(dataset_basedir, "labels.txt"), 'rt') as labelfile:
    trainset = cavitylearn.data.DataSet(labelfile, os.path.join(dataset_basedir, "boxes"), dataconfig)

In [ ]:
print('Examples available in dataset: ', trainset.N)

## Building the model

In [ ]:
boxes_placeholder = tf.placeholder(tf.float32, shape=[None, dataconfig.boxshape[0], dataconfig.boxshape[1], dataconfig.boxshape[2], dataconfig.num_props])
labels_placholder = tf.placeholder(tf.float32, shape=[None, dataconfig.num_classes])

In [ ]:
logits = cavitylearn.catalonet0.inference(boxes_placeholder, dataconfig)

In [ ]:
loss = cavitylearn.catalonet0.loss(logits, labels_placholder)

In [ ]:
global_step = tf.Variable(0, name='global_step', trainable=False)

In [ ]:
train_op = cavitylearn.catalonet0.train(loss, 1e-4, global_step)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits,1),  tf.argmax(labels_placholder,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
tf.scalar_summary("accuracy", accuracy)

## Training the Model

In [ ]:
def purge(directory, pattern):
    for f in os.listdir(directory):
        if re.search(pattern, f):
            os.remove(os.path.join(directory, f))

In [ ]:
saver = tf.train.Saver()

In [ ]:
summary_op = tf.merge_all_summaries()

In [ ]:
CONTINUE = True
BATCHSIZE = 50
NUM_BATCHES = 10
RUN_NAME = "run0"
CHECKPOINT_FREQUENCY = 20

In [ ]:
checkpoint_dir = os.path.join(dataset_basedir, "checkpoints", RUN_NAME)
trainset.rewind_batches()
with tf.Session() as sess:
    # purge log directory for run, before running it again
    purge(os.path.join(dataset_basedir, "logs", RUN_NAME), r'^events\.out\.tfevents\.\d+')
    
    # Create summary writer
    summary_writer = tf.train.SummaryWriter(os.path.join(dataset_basedir, "logs", RUN_NAME), sess.graph)
    
    
    sess.run(tf.initialize_all_variables())
    
    start_time = time.time()
    for i in range(NUM_BATCHES):
        # get training data
        labels, boxes = trainset.next_batch(BATCHSIZE)
        
        # abort training if we didn't get any more data
        if len(labels) == 0:
            break
        
        feed_dict = {
            boxes_placeholder: boxes,
            labels_placholder: labels
        }
        
        # Do it!
        sess.run(train_op, feed_dict=feed_dict)
        print('.', end="")
        
        
        summary_str, step = sess.run([summary_op, global_step], feed_dict=feed_dict)
        summary_writer.add_summary(summary_str, step)
        summary_writer.flush()
        
        if i % CHECKPOINT_FREQUENCY == 0:
            saver.save(sess, checkpoint_dir, global_step=global_step)
            
        
    saver.save(sess, checkpoint_dir, global_step=global_step)
    end_time = time.time()

In [ ]:
print("Total time: ", str(end_time - start_time))
print("Time per batch: ", str((end_time - start_time)/NUM_BATCHES))

importlib.reload(cavitylearn.catalonet0)
tf.reset_default_graph()